In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.graph_objs as go
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from datetime import date
from functools import reduce
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.metrics import silhouette_score
from scipy.stats import boxcox
import seaborn as sns
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/sales_forecasting.csv')
pd.set_option('display.max_columns', None)
data.head()

,Unnamed: 0,InvoiceID,Date,ProductID,TotalSales,Discount,CustomerID,Quantity
0,0,328,2019-12-27,1684,796.610169,143.389831,185,4
1,1,329,2019-12-27,524,355.932203,64.067797,185,2
2,2,330,2019-12-27,192,901.694915,162.305085,230,4
3,3,330,2019-12-27,218,182.754237,32.895763,230,1
4,4,330,2019-12-27,247,780.101695,140.418305,230,4


## Pre-processing

In [ ]:
sales_data = data.groupby(['Date'], as_index=False)['Quantity'].sum()
sales_data['Date'] = pd.to_datetime(sales_data['Date'],format = '%Y-%m-%d')
sales_data

,Date,Quantity
0,2019-01-02,109
1,2019-01-03,51
2,2019-01-04,65
3,2019-01-05,52
4,2019-01-07,89
...,...,...
1263,2023-03-21,85
1264,2023-03-22,137
1265,2023-03-23,58
1266,2023-03-24,48


In [ ]:
normalize = MinMaxScaler()
sales_data['Quantity'] = normalize.fit_transform(sales_data[['Quantity']])

In [ ]:
sales_data

,Date,Quantity
0,2019-01-02,0.074023
1,2019-01-03,0.034270
2,2019-01-04,0.043866
3,2019-01-05,0.034955
4,2019-01-07,0.060315
...,...,...
1263,2023-03-21,0.057574
1264,2023-03-22,0.093215
1265,2023-03-23,0.039068
1266,2023-03-24,0.032214


In [ ]:
series = sales_data.drop(['Date'], axis=1)

In [ ]:
series = np.array(series)

In [ ]:
split_size =  int(np.ceil( len(series) * .80 ))
split_size

1015

In [ ]:
# Create the scaled training data set
train_data = series[0:int(split_size), :]

# Split the data into x_train and y_train data sets
x_train = []
y_train = []

# we use 60 days ahead for forecasting range
for i in range(365, len(train_data)):
    x_train.append(train_data[i-365:i, 0])
    y_train.append(train_data[i, 0])

In [ ]:
# Convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, activation="relu", padding='causal', input_shape=[None, 1]),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, activation="relu", padding='causal'),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1),
])

model.compile(loss=tf.keras.losses.Huber(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=["mae"])
model.fit(x_train, y_train, epochs=50)

Epoch 1/50
21/21 [==============================] - 26s 956ms/step - loss: 0.0055 - mae: 0.0612
Epoch 2/50
21/21 [==============================] - 20s 976ms/step - loss: 0.0049 - mae: 0.0562
Epoch 3/50
21/21 [==============================] - 22s 1s/step - loss: 0.0049 - mae: 0.0550
Epoch 4/50
21/21 [==============================] - 23s 1s/step - loss: 0.0048 - mae: 0.0557
Epoch 5/50
21/21 [==============================] - 21s 981ms/step - loss: 0.0049 - mae: 0.0536
Epoch 6/50
21/21 [==============================] - 22s 1s/step - loss: 0.0047 - mae: 0.0543
Epoch 7/50
21/21 [==============================] - 21s 967ms/step - loss: 0.0048 - mae: 0.0534
Epoch 8/50
21/21 [==============================] - 20s 955ms/step - loss: 0.0047 - mae: 0.0529
Epoch 9/50
21/21 [==============================] - 20s 950ms/step - loss: 0.0046 - mae: 0.0537
Epoch 10/50
21/21 [==============================] - 22s 1s/step - loss: 0.0046 - mae: 0.0530
Epoch 11/50
21/21 [==============================] 